In [14]:
import json
import numpy as np
import pandas as pd

In [111]:
def append_pseudo_labeling(new_json, submission_path, score_threshold):
    submission = np.array(pd.read_csv(submission_path))
    
    for data in submission:
        if isinstance(data[0], str):

            img_id = len(new_json['images'])

            exist_box = False
            data[0] = data[0].split()
            for i in range(0, len(data[0]), 6):
                if float(data[0][i+1]) > score_threshold:
                    exist_box = True
                    anno_id = len(new_json['annotations'])
                    new_json['annotations'].append({'id': anno_id,
                                                    'image_id': img_id,
                                                    'category_id': int(data[0][i]),
                                                    'segmentation': 0,
                                                    'area': 0,
                                                    'bbox': [float(data[0][i+2]),
                                                             float(data[0][i+3]),
                                                             float(data[0][i+4])-float(data[0][i+2]),
                                                             float(data[0][i+5])-float(data[0][i+3])
                                                            ],
                                                    'iscrowd': 0
                                                   })

            if exist_box:
                new_json['images'].append({'license': 0,
                                           'url': None,
                                           'file_name': data[1],
                                           'height': 512,
                                           'width': 512,
                                           'date_captured': None,
                                           'id': img_id
                                           })

    return new_json

In [112]:
# 1. make new json
score_threshold = 0.9
submission_path = '/opt/ml/code/mmdetection_trash/Mybaseline/submissions/sm20_423.csv'

test_json = {}
with open(f'/opt/ml/input/data/test.json', 'r') as f:
    test_json.update(json.load(f))

new_json = {'info':[], 'licenses':[], 'images':[], 'categories':[], 'annotations':[]}
new_json['info'] = test_json['info']
new_json['licenses'] = test_json['licenses']
new_json['categories'] = test_json['categories']

new_json = append_pseudo_labeling(new_json, submission_path, score_threshold)

save_path = '/opt/ml/input/data/pseudo.json'
with open(save_path, 'w') as f:
    json.dump(new_json, f)

In [62]:
# 2. append to train data
score_threshold = 0.5
submission_path = '/opt/ml/code/mmdetection_trash/Mybaseline/submissions/faster_rcnn_resnet101_fpn0.1.csv'

new_json = {}
with open(f'/opt/ml/input/data/train.json', 'r') as f:
    new_json.update(json.load(f))

new_json = append_pseudo_labeling(new_json, submission_path, score_threshold)
PL_json = {'root': new_json}

save_path = '/opt/ml/input/data/pseudo.json'
with open(save_path, 'w') as f:
    json.dump(new_json, f)

KeyboardInterrupt: 